In [4]:
import json
from ollama import Client
from typing import List, Dict, Optional
from pydantic import BaseModel, Field
import json_merge_patch as jmp


# Complete tag list from original
TAG_GROUPS = {
    "TIME_DURATION": [
        "1-day-or-more", "15-minutes-or-less", "30-minutes-or-less",
        "4-hours-or-less", "60-minutes-or-less"
    ],
    "DIFFICULTY_SCALE": [
        "3-steps-or-less", "5-ingredients-or-less", "beginner-cook", "easy",
        "for-1-or-2", "for-large-groups", "one-dish-meal"
    ],
    "DIETARY_HEALTH": [
        "dairy-free", "diabetic", "egg-free", "gluten-free",
        "high-calcium", "high-fiber", "high-protein", "kosher", "lactose",
        "low-calorie", "low-carb", "low-cholesterol", "low-fat", "low-protein",
        "low-saturated-fat", "low-sodium", "no-shell-fish", "nut-free", "vegan",
        "vegetarian", "very-low-carbs"
    ],
    "CUISINES_REGIONAL": [
        "african", "american", "amish-mennonite", "angolan", "argentine", "asian",
        "australian", "austrian", "baja", "beijing", "belgian", "brazilian",
        "british-columbian", "cajun", "californian", "cambodian", "canadian",
        "cantonese", "caribbean", "central-american", "chilean", "chinese",
        "colombian", "congolese", "costa-rican", "creole", "cuban", "czech",
        "danish", "dutch", "ecuadorean", "egyptian", "english", "ethiopian",
        "european", "filipino", "finnish", "french", "georgian", "german", "greek",
        "guatemalan", "hawaiian", "honduran", "hunan", "hungarian", "icelandic",
        "indian", "indonesian", "iranian-persian", "iraqi", "irish", "italian",
        "japanese", "jewish-ashkenazi", "jewish-sephardi", "korean", "laotian",
        "lebanese", "libyan", "malaysian", "mexican", "micro-melanesia",
        "middle-eastern", "midwestern", "mongolian", "moroccan", "namibian",
        "native-american", "nepalese", "new-zealand", "nigerian", "north-american",
        "northeastern-united-states", "norwegian", "oaxacan", "ontario",
        "pacific-northwest", "pakistani", "palestinian", "pennsylvania-dutch",
        "peruvian", "polish", "polynesian", "portuguese", "puerto-rican", "quebec",
        "russian", "saudi-arabian", "scandinavian", "scottish", "somalian", "soul",
        "south-african", "south-american", "south-west-pacific",
        "southern-united-states", "southwestern-united-states", "spanish",
        "sudanese", "swedish", "swiss", "szechuan", "tex-mex", "thai", "turkish",
        "venezuelan", "vietnamese", "welsh"
    ],
    "MEAL_COURSES": [
        "appetizers", "beverages", "breakfast", "breakfast-eggs", "brewing",
        "brunch", "cocktails", "desserts", "dinner-party", "eggs-breakfast",
        "eggs-dairy", "granola-and-porridge", "lunch", "main-dish",
        "mashed-potatoes", "non-alcoholic", "punch", "shakes", "snacks"
    ],
    "PREPARATION_METHOD": [
        "baking", "barbecue", "bread-machine", "broil", "canning",
        "crock-pot-main-dish", "crock-pot-slow-cooker", "deep-fry", "dehydrator",
        "food-processor-blender", "freezer", "grilling", "microwave", "mixer",
        "no-cook", "oven", "pressure-canning", "pressure-cooker", "refrigerator",
        "roast", "small-appliance", "smoker", "steam", "stir-fry", "stove-top",
        "unprocessed-freezer", "water-bath"
    ]
}

ALL_TAGS = [tag for group in TAG_GROUPS.values() for tag in group]
VALID_TAGS = set(ALL_TAGS)

In [5]:
"""
Recipe State Manager with JSON Merge Patches
Using RFC 7386 patches and Pydantic V2 compatible
"""

# Models for full state and patches
class RecipeState(BaseModel):
    """Full recipe state"""
    name_description: str = ""
    include_tags: List[str] = Field(default_factory=list)
    exclude_tags: List[str] = Field(default_factory=list)
    include_ingredients: List[str] = Field(default_factory=list)
    exclude_ingredients: List[str] = Field(default_factory=list)
    count: int = 5
    reason: str = ""

class RecipePatch(BaseModel):
    """Patch for updates"""
    name_description: Optional[str] = None
    include_tags: Optional[List[str]] = None
    exclude_tags: Optional[List[str]] = None
    include_ingredients: Optional[List[str]] = None
    exclude_ingredients: Optional[List[str]] = None
    count: Optional[int] = None
    reason: str

# Fallback merge function
def merge_patch(target: Dict, patch: Dict) -> Dict:
    """Simple merge patch implementation"""
    result = target.copy() if target else {}
    
    for key, value in patch.items():
        if value is None:
            result.pop(key, None)
        else:
            result[key] = value
    
    return result

# Patch-based manager
class PatchManager:
    def __init__(self):
        self.client = Client()
        self.state: Optional[Dict] = None
        
    def process(self, user_input: str) -> Dict:
        # Check if this is first request or reset
        is_first = self.state is None
        is_reset = any(word in user_input.lower() for word in ["forget", "instead", "start over"])
        
        if is_first or is_reset:
            return self._process_full(user_input)
        else:
            return self._process_patch(user_input)
    
    def _process_full(self, user_input: str) -> Dict:
        """Process as full state"""
        prompt = f"""Create a recipe search JSON for: "{user_input}"

Valid tags: {', '.join(ALL_TAGS[:30])}... ({len(ALL_TAGS)} total)
If something is mentioned as quick , fast etc -> "15-minutes-or-less", "30-minutes-or-less" are the right tags to use.
Example output:
{{
  "name_description": "quick vegan lunch",
  "include_tags": ["15-minutes-or-less", "30-minutes-or-less", "vegan", "lunch"],
  "include_ingredients": [],
  "exclude_ingredients": [],
  "count": 5,
  "reason": "initial request"
}}

Food items go in ingredients, not tags.
Output JSON:"""
        
        try:
            resp = self.client.chat(
                model="llama3.2:3b",
                messages=[{"role": "user", "content": prompt}],
                format=RecipeState.model_json_schema(),
                options={"temperature": 0}
            )
            
            data = json.loads(resp.message.content)
            state = RecipeState(**data)
            
            # Convert to dict
            try:
                self.state = state.model_dump()
            except:
                self.state = state.dict()
            
            # Clean tags
            self._clean_state()
            
            return self.state
            
        except Exception as e:
            print(f"Error: {e}")
            return self.state or {}
    
    def _process_patch(self, user_input: str) -> Dict:
        """Process as patch"""
        prompt = f"""Generate a JSON patch to modify the current recipe search.

CURRENT STATE:
{json.dumps(self.state, indent=2)}

PATCH EXAMPLES:
- Add ingredient: {{"include_ingredients": ["pasta", "tomatoes"], "reason": "added pasta and tomatoes"}}
- Change count: {{"count": 8, "reason": "changed to 8 recipes"}}
- Exclude item: {{"exclude_ingredients": ["eggs", "dairy"], "reason": "no eggs or dairy"}}
- Add tag: {{"include_tags": ["italian", "easy"], "reason": "added italian and easy tags"}}
- Never add tags neither to include_tags nor to exclude_tags unless explicitly mentioned or implied 
- Never add ingredients neither to include_ingredients nor to exclude_ingredients unless explicitly mentioned or implied 
User request: "{user_input}"

Output ONLY the fields that change:"""
        
        try:
            resp = self.client.chat(
                model="llama3.2:3b",
                messages=[{"role": "user", "content": prompt}],
                format=RecipePatch.model_json_schema(),
                options={"temperature": 0}
            )
            
            patch_data = json.loads(resp.message.content)
            patch = RecipePatch(**patch_data)
            
            # Convert to dict
            try:
                patch_dict = patch.model_dump(exclude_none=True)
            except:
                patch_dict = patch.dict(exclude_none=True)
            
            # Apply patch
            if HAS_JMP:
                self.state = jmp.merge(self.state, patch_dict)
            else:
                self.state = merge_patch(self.state, patch_dict)
            
            # Clean tags
            self._clean_state()
            
            print(f"Applied patch: {patch_dict}")
            
            return self.state
            
        except Exception as e:
            print(f"Patch failed: {e}, falling back to full update")
            return self._process_full(user_input)
    
    def _clean_state(self):
        """Clean tags in current state"""
        if not self.state:
            return
            
        # Filter valid tags
        self.state['include_tags'] = [
            t for t in self.state.get('include_tags', []) 
            if t in VALID_TAGS
        ][:5]
        
        self.state['exclude_tags'] = [
            t for t in self.state.get('exclude_tags', [])
            if t in VALID_TAGS
        ][:5]

In [6]:
# Test function
def test_patch_manager():
    print(f"\nPATCH-BASED RECIPE MANAGER")
    print(f"Using {len(ALL_TAGS)} valid tags")
    print("="*60)
    
    manager = PatchManager()
    
    tests = [
        "I'd like quick vegetarian breakfasts, no eggs please",
        "Make it 6 recipes and add oats",
        "Let's forget everything and just look for a nice dinner meal with beef, beef with casserole would be good",
        "no onions please",
        "add some italian style"
    ]
    
    for i, test in enumerate(tests, 1):
        print(f"\nTest {i}: {test}")
        state = manager.process(test)
        
        print(f"Result:")
        print(f"  Description: {state.get('name_description', '')}")
        print(f"  Tags: {state.get('include_tags', [])}")
        print(f"  Ingredients: {state.get('include_ingredients', [])}")
        print(f"  Excluded: {state.get('exclude_ingredients', [])}")
        print(f"  Count: {state.get('count', 5)}")
        print(f"  Reason: {state.get('reason', '')}")

if __name__ == "__main__":
    test_patch_manager()


PATCH-BASED RECIPE MANAGER
Using 188 valid tags

Test 1: I'd like quick vegetarian breakfasts, no eggs please
Result:
  Description: Quick Vegetarian Breakfasts
  Tags: ['15-minutes-or-less', '30-minutes-or-less', 'vegetarian', 'breakfast']
  Ingredients: []
  Excluded: []
  Count: 0
  Reason: initial request}  {

Test 2: Make it 6 recipes and add oats
Applied patch: {'include_ingredients': ['oats'], 'count': 6, 'reason': 'changed to 6 recipes'}
Result:
  Description: Quick Vegetarian Breakfasts
  Tags: ['15-minutes-or-less', '30-minutes-or-less', 'vegetarian', 'breakfast']
  Ingredients: ['oats']
  Excluded: []
  Count: 6
  Reason: changed to 6 recipes

Test 3: Let's forget everything and just look for a nice dinner meal with beef, beef with casserole would be good
Result:
  Description: beef casserole
  Tags: ['4-hours-or-less', 'one-dish-meal']
  Ingredients: ['beef', 'casseroles']
  Excluded: []
  Count: 10
  Reason: initial request

Test 4: no onions please
Applied patch: {'exclu